## Importing necessary Libraries and Loading the Data

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load the data
data = pd.read_excel(r'/content/drive/MyDrive/Colab Notebooks/ANN_Assignment_3.xlsx')

In [5]:

# Extract input parameters (X1-X35) and output quality measurements (R-MF, Q-MF)
X = data.iloc[:, 1:36].values
R_MF = data.iloc[:, 18].values  # Assuming R-MF is in the 19th column
Q_MF = data.iloc[:, -2].values  # Assuming Q-MF is the second-to-last column

# Data preprocessing
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_R_MF = StandardScaler()
R_MF_scaled = scaler_R_MF.fit_transform(R_MF.reshape(-1, 1))

scaler_Q_MF = StandardScaler()
Q_MF_scaled = scaler_Q_MF.fit_transform(Q_MF.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, R_MF_train, R_MF_test, Q_MF_train, Q_MF_test = train_test_split(
    X_scaled, R_MF_scaled, Q_MF_scaled, test_size=0.2, random_state=42
)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
R_MF_train_tensor = torch.FloatTensor(R_MF_train)
Q_MF_train_tensor = torch.FloatTensor(Q_MF_train)

X_test_tensor = torch.FloatTensor(X_test)
R_MF_test_tensor = torch.FloatTensor(R_MF_test)
Q_MF_test_tensor = torch.FloatTensor(Q_MF_test)

# Define the neural network model
class SimpleANN(nn.Module):
    def __init__(self, input_size):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2_R_MF = nn.Linear(64, 1)  # Output layer for R-MF
        self.fc2_Q_MF = nn.Linear(64, 1)  # Output layer for Q-MF

    def forward(self, x):
        x = self.relu(self.fc1(x))
        R_MF_output = self.fc2_R_MF(x)
        Q_MF_output = self.fc2_Q_MF(x)
        return R_MF_output, Q_MF_output

# Instantiate the model
input_size = X_train.shape[1]
model = SimpleANN(input_size)

# Define loss functions and optimizers for R-MF and Q-MF
criterion_R_MF = nn.MSELoss()
criterion_Q_MF = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    R_MF_output, Q_MF_output = model(X_train_tensor)
    loss_R_MF = criterion_R_MF(R_MF_output, R_MF_train_tensor)
    loss_Q_MF = criterion_Q_MF(Q_MF_output, Q_MF_train_tensor)
    total_loss = loss_R_MF + loss_Q_MF

    # Backward pass and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], R_MF Loss: {loss_R_MF.item():.4f}, Q_MF Loss: {loss_Q_MF.item():.4f}')

# Testing the model
with torch.no_grad():
    model.eval()
    R_MF_test_output, Q_MF_test_output = model(X_test_tensor)
    test_loss_R_MF = criterion_R_MF(R_MF_test_output, R_MF_test_tensor)
    test_loss_Q_MF = criterion_Q_MF(Q_MF_test_output, Q_MF_test_tensor)
    print(f'Test R_MF Loss: {test_loss_R_MF.item():.4f}, Test Q_MF Loss: {test_loss_Q_MF.item():.4f}')


Epoch [100/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [200/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [300/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [400/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [500/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [600/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [700/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [800/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [900/1000], R_MF Loss: nan, Q_MF Loss: nan
Epoch [1000/1000], R_MF Loss: nan, Q_MF Loss: nan
Test R_MF Loss: nan, Test Q_MF Loss: nan
